In [67]:
import collections
from collections import defaultdict
import time
import pdb

class ROIDetector:
    def __init__(self, max_history=100, min_confidence=0.79, min_hits=5, max_distance=10):
        self.history = collections.deque(maxlen=max_history)
        self.clusters = defaultdict(lambda: {'hits': 0, 'total_confidence': 0, 'positions': []})
        self.max_history = max_history
        self.min_confidence = min_confidence
        self.min_hits = min_hits
        self.max_distance = max_distance

    def reset(self):
        self.history = collections.deque(maxlen=self.max_history)
        self.clusters = defaultdict(lambda: {'hits': 0, 'total_confidence': 0, 'positions': []})
        
    def add_roi(self, left, top, right, bottom, confidence):
        # 현재 발생한 ROI 중심점 및 크기 계산
        center_x = (left + right) / 2
        center_y = (top + bottom) / 2
        size = (right - left) * (bottom - top)

        # 적절한 클러스터 찾기
        matched = False
        for (cx, cy), data in self.clusters.items():
            if abs(center_x - cx) <= self.max_distance and abs(center_y - cy) <= self.max_distance:
                # 비슷한 클러스터에 ROI 추가
                data['hits'] += 1
                data['total_confidence'] += confidence
                data['positions'].append((center_x, center_y, size, confidence))
                matched = True
                break

        if not matched:
            # 새 클러스터 생성
            self.clusters[(center_x, center_y)]['positions'].append((center_x, center_y, size, confidence))
            self.clusters[(center_x, center_y)]['hits'] = 1
            self.clusters[(center_x, center_y)]['total_confidence'] = confidence

        # 현재 ROI 정보 추가
        self.history.append((center_x, center_y, size, confidence))
        self.check_for_detection()

    def check_for_detection(self):
        # 감지 조건을 확인하고 트리거
        for (cx, cy), data in list(self.clusters.items()):
            if data['hits'] >= self.min_hits:
                avg_confidence = data['total_confidence'] / data['hits']
                if avg_confidence >= self.min_confidence:
                    print(f"Detection triggered at center ({cx}, {cy}) with average confidence {avg_confidence}, hit: {data['hits']}")

    def clean_up(self):
        # 데이터 정리
        current_positions = set((x, y) for x, y, _, _ in self.history)
        to_remove = []
    
        for (cx, cy) in self.clusters:
            # 현재 클러스터의 중심이 최근 ROI 중심들과 비교하여 max_distance 이내에 있는지 확인
            if not any(abs(cx - x) <= self.max_distance and abs(cy - y) <= self.max_distance for x, y in current_positions):
                to_remove.append((cx, cy))
    
        # 범위 밖의 클러스터 데이터 제거
        for pos in to_remove:
            del self.clusters[pos]


# 예시 사용
detector = ROIDetector()
for ii in range(10):
    print(ii)
    # 테스트를 위한 무작위 데이터 입력
    detector.add_roi(left=10, top=10, right=30, bottom=30, confidence=0.7)
    detector.add_roi(left=15, top=15, right=35, bottom=35, confidence=0.6)
    detector.add_roi(left=400, top=300, right=420, bottom=320, confidence=0.8)
    #time.sleep(1)  # 1초 대기


print("\n")

for ii,((cx, cy), data) in enumerate(detector.clusters.items()):
    avg_confidence = data['total_confidence'] / data['hits']
    print(f"{ii}: center:({cx},{cy}), data: {data}, Conf: {avg_confidence:.3f}")
    print()


#print(detector.clean_up())
print(len(detector.history), detector.history)

0
1
2
3
4
Detection triggered at center (410.0, 310.0) with average confidence 0.8, hit: 5
5
Detection triggered at center (410.0, 310.0) with average confidence 0.8, hit: 5
Detection triggered at center (410.0, 310.0) with average confidence 0.8, hit: 5
Detection triggered at center (410.0, 310.0) with average confidence 0.7999999999999999, hit: 6
6
Detection triggered at center (410.0, 310.0) with average confidence 0.7999999999999999, hit: 6
Detection triggered at center (410.0, 310.0) with average confidence 0.7999999999999999, hit: 6
Detection triggered at center (410.0, 310.0) with average confidence 0.7999999999999999, hit: 7
7
Detection triggered at center (410.0, 310.0) with average confidence 0.7999999999999999, hit: 7
Detection triggered at center (410.0, 310.0) with average confidence 0.7999999999999999, hit: 7
Detection triggered at center (410.0, 310.0) with average confidence 0.7999999999999999, hit: 8
8
Detection triggered at center (410.0, 310.0) with average confidenc

In [92]:
import random

detector = ROIDetector()

# 예시 사용
def roi_rand():
    left1 = random.randint(0, 500)
    top1 = random.randint(0, 500)
    width1 = random.randint(20, 100)
    height1 = random.randint(20, 100)
    confidence1 = random.uniform(0.5, 1.0)
    return left1, top1, width1, height1, confidence1
    
for ii in range(300):
    # 테스트를 위한 무작위 데이터 입력
    left1, top1, width1, height1, confidence1 = roi_rand()
    detector.add_roi(left=left1, top=top1, right=left1 + width1, bottom=top1 + height1, confidence=confidence1)

    left2, top2, width2, height2, confidence2 = roi_rand()    
    detector.add_roi(left=left2, top=top2, right=left2 + width2, bottom=top2 + height2, confidence=confidence2)

    left3, top3, width3, height3, confidence3 = roi_rand()    
    detector.add_roi(left=left3, top=top3, right=left3 + width3, bottom=top3 + height3, confidence=confidence3)

Detection triggered at center (498.5, 150.5) with average confidence 0.8953375194089285, hit: 5
Detection triggered at center (498.5, 150.5) with average confidence 0.8953375194089285, hit: 5
Detection triggered at center (498.5, 150.5) with average confidence 0.8953375194089285, hit: 5
Detection triggered at center (498.5, 150.5) with average confidence 0.8953375194089285, hit: 5
Detection triggered at center (498.5, 150.5) with average confidence 0.8953375194089285, hit: 5
Detection triggered at center (498.5, 150.5) with average confidence 0.8953375194089285, hit: 5
Detection triggered at center (498.5, 150.5) with average confidence 0.8953375194089285, hit: 5
Detection triggered at center (498.5, 150.5) with average confidence 0.8953375194089285, hit: 5
Detection triggered at center (498.5, 150.5) with average confidence 0.8953375194089285, hit: 5
Detection triggered at center (498.5, 150.5) with average confidence 0.8953375194089285, hit: 5
Detection triggered at center (498.5, 15

In [93]:
detector.clean_up()
#detector.reset()

for ii,((cx, cy), data) in enumerate(detector.clusters.items()):
    avg_confidence = data['total_confidence'] / data['hits']
    print(f"{ii}: center:({cx},{cy}), data: {data}, Conf: {avg_confidence:.3f}")
    print()


#print(detector.clean_up())
print(len(detector.history), detector.history)

0: center:(148.0,116.5), data: {'hits': 4, 'total_confidence': 3.0557523738272057, 'positions': [(148.0, 116.5, 2490, 0.7831486463710959), (143.0, 126.0, 7568, 0.8341250016831866), (146.0, 107.5, 4160, 0.7028208857051526), (146.0, 116.0, 1440, 0.7356578400677709)]}, Conf: 0.764

1: center:(59.0,491.0), data: {'hits': 2, 'total_confidence': 1.7266618950836936, 'positions': [(59.0, 491.0, 1064, 0.7751830245464923), (56.5, 498.5, 2303, 0.9514788705372013)]}, Conf: 0.863

2: center:(294.0,420.0), data: {'hits': 3, 'total_confidence': 2.2087103406118196, 'positions': [(294.0, 420.0, 560, 0.6158637763651784), (302.5, 425.5, 2795, 0.8063811358768502), (302.0, 428.5, 1340, 0.7864654283697909)]}, Conf: 0.736

3: center:(73.0,440.5), data: {'hits': 4, 'total_confidence': 3.1234868577613817, 'positions': [(73.0, 440.5, 4690, 0.5389330023210177), (82.5, 450.5, 3315, 0.925959348006755), (69.0, 438.0, 8648, 0.9619878257117804), (74.5, 431.5, 5265, 0.6966066817218286)]}, Conf: 0.781

4: center:(104.5